In [70]:
import sys
import os

# os.getcwd() => '/home/genie/Documents/Projekti/cash-flow/jupyter'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Moves up one level to 'cash-flow'
# project_root = '/home/genie/Documents/Projekti/cash-flow'

# Add the module Converters directory to sys.path
sys.path.append(os.path.join(project_root, 'cash_flow', 'util'))



In [71]:
# Imports and engine

import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine

from Converters import date_format

pd.options.mode.copy_on_write = True
engine = create_engine("sqlite:///../data/database.db", echo=False)

In [72]:
# -------------------
# CONFIGURABLE INPUTS
# -------------------

date_from = "2024-01-01"
date_through = "2025-12-31"

# Choose period granularity: "day", "week", "month", "quarter", "year"
period = "month"


In [73]:
# Mapping period to offset (end of period)
period_offsets = {
    "day": pd.offsets.Day(0),
    "week": pd.offsets.Week(weekday=6),         # Sunday
    "month": pd.offsets.MonthEnd(0),
    "quarter": pd.offsets.QuarterEnd(startingMonth=12),
    "year": pd.offsets.YearEnd(0)
}
date_offset = period_offsets.get(period, pd.offsets.MonthEnd(0))
date_freq = {
    "day": "D",
    "week": "W-SUN",         # Sunday
    "month": "ME",
    "quarter": "QE",
    "year": "YE"
}
date_frequency = date_freq.get(period, "ME")

In [74]:
# Load all data from database

actual = pd.read_sql_query('SELECT * FROM G10_CashFlow_Actual_Corresponding WHERE d_date >= "' + date_from + '" AND d_date <= "' + date_through + '" ', engine)
pending = pd.read_sql_query('SELECT * FROM G12_CashFlow_Pending_Corresponding WHERE p_date >= "' + date_from + '" AND p_date <= "' + date_through + '" ', engine)
budgeted = pd.read_sql_query('SELECT * FROM F01_BudgetEntries WHERE date >= "' + date_from + '" AND date <= "' + date_through + '" ', engine)
cash = pd.read_sql_query('SELECT * FROM G02_CashTransactions WHERE d_date <= "' + date_through + '" ', engine)
definition_df = pd.read_sql_table("E01_CashFlowDefinition", engine)

definition_accounts_df = pd.read_sql_table("E01_CashFlowDefinitionAccounts", engine)
definition_totals_df = pd.read_sql_table("E01_CashFlowDefinitionTotals", engine)



In [75]:
# Ensure required columns and formats

actual['period_end'] = pd.to_datetime(actual['d_date'], format='mixed') + date_offset
actual['cf_amount'] = np.where(actual['gl_entry_type'] == 'CR', actual['gl_amount_LC'], -actual['gl_amount_LC'])
pending['period_end'] = pd.to_datetime(pending['p_date'], format='mixed') + date_offset
pending['cf_amount'] = np.where(pending['gl_entry_type'] == 'CR', pending['gl_amount_LC'], -pending['gl_amount_LC'])
budgeted['period_end'] = pd.to_datetime(budgeted['date'], format='mixed') + date_offset
budgeted['cf_amount'] = np.where(budgeted['cash_type'] == 'Receipt', budgeted['amount_LC'], -budgeted['amount_LC'])
cash['period_end'] = pd.to_datetime(cash['d_date'], format='mixed') + date_offset
cash['cf_amount'] = np.where(cash['gl_entry_type'] == 'DR', cash['gl_amount_LC'], -cash['gl_amount_LC'])

definition_df.rename(columns={"id":"definition_id"}, inplace=True)
definition_acc_df = definition_df[definition_df["definition_type"] == 1]
definition_tot_df = definition_df[definition_df["definition_type"] == 2]
definition_bal_df = definition_df[definition_df["definition_type"] == 3]

In [76]:
actual.head()

,d_id,cash_status,cash_type,d_type,d_date,d_number,d_customer_id,d_vendor_ir,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,4736,Actual,Payment,2,2024-01-01 00:00:00.000000,3436,NaN,36.0,Payment 3436,EUR,DR,5721,941.49,941.49,2024-01-31,-941.49
1,4736,Actual,Payment,2,2024-01-01 00:00:00.000000,3436,NaN,36.0,Payment 3436,EUR,DR,7110,293.67,293.67,2024-01-31,-293.67
2,4736,Actual,Payment,2,2024-01-01 00:00:00.000000,3436,NaN,36.0,Payment 3436,EUR,DR,7210,2556.68,2556.68,2024-01-31,-2556.68
3,4736,Actual,Payment,2,2024-01-01 00:00:00.000000,3436,NaN,36.0,Payment 3436,EUR,DR,7510,1632.93,1632.93,2024-01-31,-1632.93
4,2254,Actual,Receipt,1,2024-01-02 00:00:00.000000,1754,50.0,NaN,Payment 1754,EUR,CR,5721,1527.91,1527.91,2024-01-31,1527.91


In [77]:
pending.head()

,d_id,cash_status,cash_type,d_type,p_date,d_number,d_customer_id,d_vendor_id,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,8,Pending,Receipt,3,2025-05-12,8,39.0,NaN,Invoice 8,EUR,CR,6110,601.70,601.70,2025-05-31,601.70
1,8,Pending,Receipt,3,2025-05-12,8,39.0,NaN,Invoice 8,EUR,CR,5721,126.36,126.36,2025-05-31,126.36
2,11,Pending,Receipt,3,2025-05-12,11,49.0,NaN,Invoice 11,EUR,CR,6110,2833.28,2833.28,2025-05-31,2833.28
3,11,Pending,Receipt,3,2025-05-12,11,49.0,NaN,Invoice 11,EUR,CR,5721,594.99,594.99,2025-05-31,594.99
4,15,Pending,Receipt,3,2025-05-12,15,29.0,NaN,Invoice 15,EUR,CR,6550,8966.10,8966.10,2025-05-31,8966.10


In [78]:
budgeted.head()

,id,definition_id,cash_type,date,amount_LC,memo,period_end,cf_amount
0,1,2,Receipt,2025-05-04 00:00:00.000000,1000,Realizācija,2025-05-31,1000
1,2,2,Receipt,2025-05-11 00:00:00.000000,1000,Realizācija,2025-05-31,1000
2,3,2,Receipt,2025-05-18 00:00:00.000000,1000,Realizācija,2025-05-31,1000
3,4,2,Receipt,2025-05-25 00:00:00.000000,1000,Realizācija,2025-05-31,1000
4,5,2,Receipt,2025-06-01 00:00:00.000000,1000,Realizācija,2025-06-30,1000


In [79]:
# -------------------------------------------------------------------
# ************** (1) CashFlow based on accounts definitions *********
# -------------------------------------------------------------------

#  Prepare chart of CF definitions 

# Merge accounts into accounts definition
definition_accounts_df = pd.merge(
    definition_acc_df,
    definition_accounts_df,
    on="definition_id", how="left")
definition_accounts_df.drop(columns=["id"], inplace=True)

In [80]:
definition_accounts_df

,definition_id,key,definition_type,name,cash_type,account
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110
1,3,102,1.0,Maksājumi piegādātājiem,Payment,7110
2,3,102,1.0,Maksājumi piegādātājiem,Payment,7310
3,4,103,1.0,Maksājumi darbiniekiem,Payment,7210
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,6550
5,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,7510
6,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,1210
7,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,2310
8,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,5310
9,7,201,1.0,Izdevumi procentu maksājumiem,NaN,NaN


In [81]:
actual = pd.merge(definition_accounts_df,
    actual,
    left_on=['cash_type', 'account'],        # Columns in definition_df
    right_on=['cash_type', 'gl_account'], # Corresponding columns in transactions_df
    how='left'
)


In [82]:
actual.head()

,definition_id,key,definition_type,name,cash_type,account,d_id,cash_status,d_type,d_date,...,d_customer_id,d_vendor_ir,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2254.0,Actual,1.0,2024-01-02 00:00:00.000000,...,50.0,NaN,Payment 1754,EUR,CR,6110,1821.15,1821.15,2024-01-31,1821.15
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,1907.0,Actual,1.0,2024-01-03 00:00:00.000000,...,76.0,NaN,Payment 1407,EUR,CR,6110,1108.17,1108.17,2024-01-31,1108.17
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2415.0,Actual,1.0,2024-01-03 00:00:00.000000,...,31.0,NaN,Payment 1915,EUR,CR,6110,1795.30,1795.30,2024-01-31,1795.30
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2311.0,Actual,1.0,2024-01-08 00:00:00.000000,...,9.0,NaN,Payment 1811,EUR,CR,6110,3725.53,3725.53,2024-01-31,3725.53
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2193.0,Actual,1.0,2024-01-12 00:00:00.000000,...,95.0,NaN,Payment 1693,EUR,CR,6110,3744.23,3744.23,2024-01-31,3744.23


In [83]:
pending = pd.merge(definition_accounts_df,
    pending,
    left_on=['cash_type', 'account'],        # Columns in definition_df
    right_on=['cash_type', 'gl_account'], # Corresponding columns in transactions_df
    how='left'
)


In [84]:
pending.head()

,definition_id,key,definition_type,name,cash_type,account,d_id,cash_status,d_type,p_date,...,d_customer_id,d_vendor_id,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,8.0,Pending,3.0,2025-05-12,...,39.0,NaN,Invoice 8,EUR,CR,6110,601.70,601.70,2025-05-31,601.70
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,11.0,Pending,3.0,2025-05-12,...,49.0,NaN,Invoice 11,EUR,CR,6110,2833.28,2833.28,2025-05-31,2833.28
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,19.0,Pending,3.0,2025-05-12,...,88.0,NaN,Invoice 19,EUR,CR,6110,3426.53,3426.53,2025-05-31,3426.53
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,31.0,Pending,3.0,2025-05-12,...,41.0,NaN,Invoice 31,EUR,CR,6110,1020.02,1020.02,2025-05-31,1020.02
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,44.0,Pending,3.0,2025-05-12,...,58.0,NaN,Invoice 44,EUR,CR,6110,224.62,224.62,2025-05-31,224.62


In [85]:
budgeted = pd.merge(definition_acc_df,
    budgeted,
    left_on=['definition_id'],        # Columns in definition_df
    right_on=['definition_id'], # Corresponding columns in transactions_df
    how='left'
)


In [86]:
budgeted.head()

,definition_id,key,definition_type,name,id,cash_type,date,amount_LC,memo,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,1.0,Receipt,2025-05-04 00:00:00.000000,1000.0,Realizācija,2025-05-31,1000.0
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,2.0,Receipt,2025-05-11 00:00:00.000000,1000.0,Realizācija,2025-05-31,1000.0
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,3.0,Receipt,2025-05-18 00:00:00.000000,1000.0,Realizācija,2025-05-31,1000.0
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,4.0,Receipt,2025-05-25 00:00:00.000000,1000.0,Realizācija,2025-05-31,1000.0
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,5.0,Receipt,2025-06-01 00:00:00.000000,1000.0,Realizācija,2025-06-30,1000.0


In [87]:
# -------------------
# GROUP & MERGE
# -------------------
def get_period_totals(df, label):
    grouped = df.groupby(['definition_id', 'period_end', 'cash_type'])['cf_amount'].sum().unstack(fill_value=0)
    columns = ["Receipt", "Payment"]
    grouped = grouped.reindex(columns=columns, fill_value=0)
    grouped.columns = [f"{label}_{col}" for col in grouped.columns]
    return grouped

In [88]:
actual_period = get_period_totals(actual, 'Actual')
pending_period = get_period_totals(pending, 'Pending')
budgeted_period = get_period_totals(budgeted, 'Budgeted')

In [89]:
# Combine all
combined = actual_period.join(pending_period, how='outer') \
                        .join(budgeted_period, how='outer') \
                        .fillna(0).reset_index()

In [90]:
combined.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment
0,2,2024-01-31,34921.83,0.0,0.0,0.0,0.0,0.0
1,2,2024-02-29,31834.94,0.0,0.0,0.0,0.0,0.0
2,2,2024-03-31,14721.51,0.0,0.0,0.0,0.0,0.0
3,2,2024-04-30,28475.07,0.0,0.0,0.0,0.0,0.0
4,2,2024-05-31,26952.20,0.0,0.0,0.0,0.0,0.0


In [91]:
# -------------------
# SPLIT PAST / FUTURE
# -------------------
today = pd.to_datetime(datetime.date.today())
this_period_end = today + date_offset

# Split
past = combined[combined['period_end'] < this_period_end]
future = combined[combined['period_end'] >= this_period_end]


In [92]:
# -------------------
# CASHFLOW LOGIC
# -------------------

# For past: use only actuals
past['income'] = past.get('Actual_Receipt', 0)
past['expense'] = past.get('Actual_Payment', 0)

In [93]:
past.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,income,expense
0,2,2024-01-31,34921.83,0.0,0.0,0.0,0.0,0.0,34921.83,0.0
1,2,2024-02-29,31834.94,0.0,0.0,0.0,0.0,0.0,31834.94,0.0
2,2,2024-03-31,14721.51,0.0,0.0,0.0,0.0,0.0,14721.51,0.0
3,2,2024-04-30,28475.07,0.0,0.0,0.0,0.0,0.0,28475.07,0.0
4,2,2024-05-31,26952.20,0.0,0.0,0.0,0.0,0.0,26952.20,0.0


In [94]:
# For future: use max(budgeted, actual+pending)
future['actual_plus_pending_income'] = future.get('Actual_Receipt', 0) + future.get('Pending_Receipt', 0)
future['actual_plus_pending_expense'] = future.get('Actual_Payment', 0) + future.get('Pending_Payment', 0)

future['income'] = future[['Budgeted_Receipt', 'actual_plus_pending_income']].max(axis=1)
future['expense'] = future[['Budgeted_Payment', 'actual_plus_pending_expense']].min(axis=1)



In [95]:
future.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,actual_plus_pending_income,actual_plus_pending_expense,income,expense
16,2,2025-05-31,41088.99,0.0,810680.55,0.0,4000.0,0.0,851769.54,0.0,851769.54,0.0
17,2,2025-06-30,21011.12,0.0,36553.27,0.0,5000.0,0.0,57564.39,0.0,57564.39,0.0
18,2,2025-07-31,7289.03,0.0,21992.93,0.0,4000.0,0.0,29281.96,0.0,29281.96,0.0
19,2,2025-08-31,21163.81,0.0,61398.90,0.0,5000.0,0.0,82562.71,0.0,82562.71,0.0
20,2,2025-09-30,21331.78,0.0,6094.69,0.0,4000.0,0.0,27426.47,0.0,27426.47,0.0


In [96]:
# Combine
cashflow = pd.concat([past, future], ignore_index=True)
cashflow['net_cashflow'] = cashflow['income'] + cashflow['expense']

In [97]:
cashflow

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,income,expense,actual_plus_pending_income,actual_plus_pending_expense,net_cashflow
0,2,2024-01-31,34921.83,0.00,0.00,0.00,0.0,0.0,34921.83,0.00,NaN,NaN,34921.83
1,2,2024-02-29,31834.94,0.00,0.00,0.00,0.0,0.0,31834.94,0.00,NaN,NaN,31834.94
2,2,2024-03-31,14721.51,0.00,0.00,0.00,0.0,0.0,14721.51,0.00,NaN,NaN,14721.51
3,2,2024-04-30,28475.07,0.00,0.00,0.00,0.0,0.0,28475.07,0.00,NaN,NaN,28475.07
4,2,2024-05-31,26952.20,0.00,0.00,0.00,0.0,0.0,26952.20,0.00,NaN,NaN,26952.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,8,2025-08-31,8342.26,-8812.14,18713.86,-10795.33,0.0,0.0,27056.12,-19607.47,27056.12,-19607.47,7448.65
116,8,2025-09-30,10277.19,-17299.56,8145.33,-5950.20,0.0,0.0,18422.52,-23249.76,18422.52,-23249.76,-4827.24
117,8,2025-10-31,12161.99,-9397.15,15265.61,-17227.41,0.0,0.0,27427.60,-26624.56,27427.60,-26624.56,803.04
118,8,2025-11-30,5241.08,-4514.49,9951.37,-18399.32,0.0,0.0,15192.45,-22913.81,15192.45,-22913.81,-7721.36


In [98]:
# -------------------
# CASHFLOW OUTPUT
# -------------------

# Pivot first
pivot_cf = cashflow.pivot_table(
    index='definition_id',
    columns='period_end',
    values='net_cashflow',
    aggfunc='sum'
).fillna(0)


In [99]:
# Fill missing periods

# Build full period range
min_period = cashflow['period_end'].min()
max_period = cashflow['period_end'].max()

all_periods = pd.date_range(start=min_period, end=max_period, freq=date_frequency)

# Reindex pivot to include all periods
pivot_cf = pivot_cf.reindex(columns=all_periods, fill_value=0)

# Sort columns just in case
pivot_cf = pivot_cf.sort_index(axis=1)

In [100]:
pivot_cf

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
2,34921.83,31834.94,14721.51,28475.07,26952.20,12450.84,19756.82,38956.14,12296.28,31397.48,...,21998.32,23706.03,851769.54,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-22915.32,-32743.86,-38358.20,-26602.02,-33158.70,-29084.71,-20624.90,-30391.87,-25157.15,-21022.10,...,-22493.64,-14946.32,-1015003.76,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,-6539.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,38367.43,5376.52,32179.48,-5840.52,-6319.27,26553.44,5564.85,-13965.61,-8228.30,-7061.08,...,-4170.01,14226.22,489998.67,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
8,7942.70,10.59,-6696.15,282.73,-8174.86,-5077.07,1194.93,-1601.22,-4283.48,1970.70,...,-3101.48,-2329.83,-25552.44,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73


In [109]:
# Store pivot_cf for cash flow figure
graph_pivot = pd.merge(definition_acc_df, pivot_cf, on="definition_id", how="right")
graph_pivot.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
graph_pivot=graph_pivot.set_index("name")

In [110]:
graph_pivot

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
name,,,,,,,,,,,,,,,,,,,,,
Ieņēmumi no preču un pakalpojumu pārdošanas,34921.83,31834.94,14721.51,28475.07,26952.20,12450.84,19756.82,38956.14,12296.28,31397.48,...,21998.32,23706.03,851769.54,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
Maksājumi piegādātājiem,-22915.32,-32743.86,-38358.20,-26602.02,-33158.70,-29084.71,-20624.90,-30391.87,-25157.15,-21022.10,...,-22493.64,-14946.32,-1015003.76,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
Maksājumi darbiniekiem,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,-6539.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
Pārējie pamatdarbības ieņēmumi un izdevumi,38367.43,5376.52,32179.48,-5840.52,-6319.27,26553.44,5564.85,-13965.61,-8228.30,-7061.08,...,-4170.01,14226.22,489998.67,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
Izdevumi nodokļu maksājumiem,7942.70,10.59,-6696.15,282.73,-8174.86,-5077.07,1194.93,-1601.22,-4283.48,1970.70,...,-3101.48,-2329.83,-25552.44,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73


In [34]:
# Ensure 0 instead of NaN in empty cells
pivot_cf = pd.merge(definition_acc_df["definition_id"], pivot_cf, left_on="definition_id", right_on="definition_id", how="left").fillna(0)
pivot_cf.set_index("definition_id", inplace=True)

In [35]:
pivot_cf

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
2,34921.83,31834.94,14721.51,28475.07,26952.20,12450.84,19756.82,38956.14,12296.28,31397.48,...,21998.32,23706.03,851769.54,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-22915.32,-32743.86,-38358.20,-26602.02,-33158.70,-29084.71,-20624.90,-30391.87,-25157.15,-21022.10,...,-22493.64,-14946.32,-1015003.76,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,-6539.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,38367.43,5376.52,32179.48,-5840.52,-6319.27,26553.44,5564.85,-13965.61,-8228.30,-7061.08,...,-4170.01,14226.22,489998.67,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,7942.70,10.59,-6696.15,282.73,-8174.86,-5077.07,1194.93,-1601.22,-4283.48,1970.70,...,-3101.48,-2329.83,-25552.44,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [36]:
# -----------------------------------------------------------------------------------------------------------------
# ********************** (2) Start working with totals based on totaled definitions *******************************
# -----------------------------------------------------------------------------------------------------------------

# Merge totals into totals definition
definition_totals_df = pd.merge(
    definition_tot_df,
    definition_totals_df,
    on="definition_id", how="left")
definition_totals_df.drop(columns=["id"], inplace=True)


In [37]:
definition_totals_df

,definition_id,key,definition_type,name,definition_summarized
0,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,2
1,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,3
2,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,4
3,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,5
4,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,2
5,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,3
6,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,4
7,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,5
8,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,7
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,8


In [38]:
# Merge totals definition with summarized accounts pivot

merged_totals = pd.merge(definition_totals_df,
    pivot_cf,
    left_on='definition_summarized',
    right_on='definition_id', 
    how='left'
)

# Drop unnecessary columns

merged_totals.drop(columns = ['key', 'definition_type', 'name', 'definition_summarized'], inplace=True)


In [39]:
merged_totals

,definition_id,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00,2024-05-31 00:00:00,2024-06-30 00:00:00,2024-07-31 00:00:00,2024-08-31 00:00:00,2024-09-30 00:00:00,...,2025-03-31 00:00:00,2025-04-30 00:00:00,2025-05-31 00:00:00,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
0,6,34921.83,31834.94,14721.51,28475.07,26952.20,12450.84,19756.82,38956.14,12296.28,...,21998.32,23706.03,851769.54,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
1,6,-22915.32,-32743.86,-38358.20,-26602.02,-33158.70,-29084.71,-20624.90,-30391.87,-25157.15,...,-22493.64,-14946.32,-1015003.76,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
2,6,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
3,6,38367.43,5376.52,32179.48,-5840.52,-6319.27,26553.44,5564.85,-13965.61,-8228.30,...,-4170.01,14226.22,489998.67,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
4,10,34921.83,31834.94,14721.51,28475.07,26952.20,12450.84,19756.82,38956.14,12296.28,...,21998.32,23706.03,851769.54,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
5,10,-22915.32,-32743.86,-38358.20,-26602.02,-33158.70,-29084.71,-20624.90,-30391.87,-25157.15,...,-22493.64,-14946.32,-1015003.76,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
6,10,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
7,10,38367.43,5376.52,32179.48,-5840.52,-6319.27,26553.44,5564.85,-13965.61,-8228.30,...,-4170.01,14226.22,489998.67,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
8,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,10,7942.70,10.59,-6696.15,282.73,-8174.86,-5077.07,1194.93,-1601.22,-4283.48,...,-3101.48,-2329.83,-25552.44,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73


In [40]:
# Choose value_columns for further summarization

value_columns = [col for col in merged_totals.columns if col not in ['definition_id']]

In [41]:
value_columns

[Timestamp('2024-01-31 00:00:00'),
 Timestamp('2024-02-29 00:00:00'),
 Timestamp('2024-03-31 00:00:00'),
 Timestamp('2024-04-30 00:00:00'),
 Timestamp('2024-05-31 00:00:00'),
 Timestamp('2024-06-30 00:00:00'),
 Timestamp('2024-07-31 00:00:00'),
 Timestamp('2024-08-31 00:00:00'),
 Timestamp('2024-09-30 00:00:00'),
 Timestamp('2024-10-31 00:00:00'),
 Timestamp('2024-11-30 00:00:00'),
 Timestamp('2024-12-31 00:00:00'),
 Timestamp('2025-01-31 00:00:00'),
 Timestamp('2025-02-28 00:00:00'),
 Timestamp('2025-03-31 00:00:00'),
 Timestamp('2025-04-30 00:00:00'),
 Timestamp('2025-05-31 00:00:00'),
 Timestamp('2025-06-30 00:00:00'),
 Timestamp('2025-07-31 00:00:00'),
 Timestamp('2025-08-31 00:00:00'),
 Timestamp('2025-09-30 00:00:00'),
 Timestamp('2025-10-31 00:00:00'),
 Timestamp('2025-11-30 00:00:00'),
 Timestamp('2025-12-31 00:00:00')]

In [42]:
# Summarize value_columns based on group value

summarized_totals = merged_totals.groupby('definition_id', group_keys=False)[value_columns].sum()

# .reset_index()

summarized_totals.fillna(0, inplace=True)

In [43]:
summarized_totals

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
6,44135.26,-6112.75,-2991.38,-10954.94,-55493.37,-12450.39,1352.97,-11957.24,-27897.54,-3225.07,...,-11754.92,7493.34,-129470.52,56739.93,13118.29,25650.31,-25379.28,-416.72,-18580.32,53707.27
10,52077.96,-6102.16,-9687.53,-10672.21,-63668.23,-17527.46,2547.90,-13558.46,-32181.02,-1254.37,...,-14856.40,5163.51,-155022.96,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00
20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
27,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
29,52077.96,-6102.16,-9687.53,-10672.21,-63668.23,-17527.46,2547.90,-13558.46,-32181.02,-1254.37,...,-14856.40,5163.51,-155022.96,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00


In [44]:
# ------------------------------------------------------------------------------------------------------------
# ********************** (3) Start working with balances on end of each period *******************************
# ------------------------------------------------------------------------------------------------------------

# ----------------------------
# STEP 1: Prepare actual_bank
# ----------------------------

# Group and sort actual bank balance by period
cash_by_period = cash.groupby('period_end')['cf_amount'].sum().sort_index()
cumulative_cash = cash_by_period.cumsum()

In [45]:
cash_by_period

period_end
2020-01-31    39610.87
2020-02-29    37597.24
2020-03-31     7935.23
2020-04-30    63097.28
2020-05-31     7497.01
                ...   
2025-08-31   -12526.88
2025-09-30   -42854.71
2025-10-31    11690.03
2025-11-30    22374.57
2025-12-31    66650.40
Name: cf_amount, Length: 72, dtype: float64

In [46]:
cumulative_cash

period_end
2020-01-31     39610.87
2020-02-29     77208.11
2020-03-31     85143.34
2020-04-30    148240.62
2020-05-31    155737.63
                ...    
2025-08-31    102471.37
2025-09-30     59616.66
2025-10-31     71306.69
2025-11-30     93681.26
2025-12-31    160331.66
Name: cf_amount, Length: 72, dtype: float64

In [47]:
# ----------------------------
# STEP 2: Determine cutoff
# ----------------------------

# All period_end columns from pivot_cf
all_periods = pivot_cf.columns.sort_values()

# Identify past and future periods
past_periods = all_periods[all_periods < this_period_end]
future_periods = all_periods[all_periods >= this_period_end]

In [48]:
# ----------------------------
# STEP 3: Build closing balance series
# ----------------------------

# Closing balance for past: from cumulative actual bank
past_closing = cumulative_cash.reindex(past_periods, method='ffill').fillna(0)

In [49]:
past_closing

2024-01-31 00:00:00    195091.93
2024-02-29 00:00:00    188989.78
2024-03-31 00:00:00    179302.25
2024-04-30 00:00:00    168630.03
2024-05-31 00:00:00    104961.78
2024-06-30 00:00:00     87434.30
2024-07-31 00:00:00     89982.19
2024-08-31 00:00:00     76423.71
2024-09-30 00:00:00     44242.68
2024-10-31 00:00:00     42988.32
2024-11-30 00:00:00    102257.10
2024-12-31 00:00:00     53257.17
2025-01-31 00:00:00     93054.80
2025-02-28 00:00:00     98845.83
2025-03-31 00:00:00     83989.40
2025-04-30 00:00:00     89152.92
Name: cf_amount, dtype: float64

In [50]:
# Starting point for future: last known balance
last_past_balance = past_closing.iloc[-1] if not past_closing.empty else 0

In [51]:
last_past_balance

89152.92

In [52]:
# Future cashflows from pivot_cf
cashflow_by_period = pivot_cf.sum(axis=0)

In [53]:
# Compute future balances
future_closing = {}
balance = last_past_balance
for period in future_periods:
    balance += cashflow_by_period.get(period, 0)
    future_closing[period] = balance

In [54]:
# Combine both into full closing balance series
full_closing_balance = pd.Series(dtype=float)
full_closing_balance = pd.concat([
    past_closing,
    pd.Series(future_closing)
]).reindex(all_periods, fill_value=0)


In [55]:
full_closing_balance

2024-01-31 00:00:00    195091.93
2024-02-29 00:00:00    188989.78
2024-03-31 00:00:00    179302.25
2024-04-30 00:00:00    168630.03
2024-05-31 00:00:00    104961.78
2024-06-30 00:00:00     87434.30
2024-07-31 00:00:00     89982.19
2024-08-31 00:00:00     76423.71
2024-09-30 00:00:00     44242.68
2024-10-31 00:00:00     42988.32
2024-11-30 00:00:00    102257.10
2024-12-31 00:00:00     53257.17
2025-01-31 00:00:00     93054.80
2025-02-28 00:00:00     98845.83
2025-03-31 00:00:00     83989.40
2025-04-30 00:00:00     89152.92
2025-05-31 00:00:00    -65870.04
2025-06-30 00:00:00     -6158.93
2025-07-31 00:00:00      4259.28
2025-08-31 00:00:00     37358.24
2025-09-30 00:00:00      7151.72
2025-10-31 00:00:00      7538.04
2025-11-30 00:00:00    -18763.64
2025-12-31 00:00:00     38437.36
dtype: float64

In [56]:
# Create a new DataFrame with closing balances only
balances = pd.DataFrame(
    [full_closing_balance],  # one row
    index=[0]  # index = 0 
)

# Create a new DataFrame with repeated rows for each definition_id
balances = pd.merge(definition_bal_df["definition_id"], balances, how='cross')
balances = balances.set_index("definition_id")


In [57]:
balances

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
30,195091.93,188989.78,179302.25,168630.03,104961.78,87434.3,89982.19,76423.71,44242.68,42988.32,...,83989.4,89152.92,-65870.04,-6158.93,4259.28,37358.24,7151.72,7538.04,-18763.64,38437.36


In [58]:
# ---------------------------------------------------------------------------------------------------------------------------------------
# ********************** (4) Put together summarized accounts, totals and balances on end of each period *******************************
# ---------------------------------------------------------------------------------------------------------------------------------------



In [65]:
pivot_cf

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
2,34921.83,31834.94,14721.51,28475.07,26952.20,12450.84,19756.82,38956.14,12296.28,31397.48,...,21998.32,23706.03,851769.54,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-22915.32,-32743.86,-38358.20,-26602.02,-33158.70,-29084.71,-20624.90,-30391.87,-25157.15,-21022.10,...,-22493.64,-14946.32,-1015003.76,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,-6539.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,38367.43,5376.52,32179.48,-5840.52,-6319.27,26553.44,5564.85,-13965.61,-8228.30,-7061.08,...,-4170.01,14226.22,489998.67,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,7942.70,10.59,-6696.15,282.73,-8174.86,-5077.07,1194.93,-1601.22,-4283.48,1970.70,...,-3101.48,-2329.83,-25552.44,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [66]:
summarized_totals

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
6,44135.26,-6112.75,-2991.38,-10954.94,-55493.37,-12450.39,1352.97,-11957.24,-27897.54,-3225.07,...,-11754.92,7493.34,-129470.52,56739.93,13118.29,25650.31,-25379.28,-416.72,-18580.32,53707.27
10,52077.96,-6102.16,-9687.53,-10672.21,-63668.23,-17527.46,2547.90,-13558.46,-32181.02,-1254.37,...,-14856.40,5163.51,-155022.96,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00
20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
27,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
29,52077.96,-6102.16,-9687.53,-10672.21,-63668.23,-17527.46,2547.90,-13558.46,-32181.02,-1254.37,...,-14856.40,5163.51,-155022.96,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00


In [67]:
balances

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
30,195091.93,188989.78,179302.25,168630.03,104961.78,87434.3,89982.19,76423.71,44242.68,42988.32,...,83989.4,89152.92,-65870.04,-6158.93,4259.28,37358.24,7151.72,7538.04,-18763.64,38437.36


In [249]:
# Concatenate it all together
report = pd.concat([pivot_cf, summarized_totals, balances])

In [250]:
report

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
2,50231.12,42880.14,26308.59,38379.57,39246.76,19862.60,28071.46,48333.03,19896.11,41910.53,...,29564.02,31612.94,1223518.79,78614.91,50383.77,109618.83,45848.99,79478.46,64314.40,78589.64
3,-30281.91,-43778.47,-56641.43,-36223.79,-53628.12,-41573.54,-27744.61,-41369.98,-37040.46,-29564.45,...,-33160.82,-25183.06,-1412305.45,-53911.59,-71354.35,-65696.15,-57152.91,-83340.54,-87955.73,-85636.35
4,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,-6539.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,39997.26,11550.30,-3411.93,6743.59,2071.27,9750.12,11096.88,-11234.59,-5011.91,7518.95,...,-10285.45,-6691.37,472238.27,32183.35,23817.56,23586.95,16967.53,52095.60,6595.89,50901.75
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [251]:
# -------------------------------------
# Prepare report for visual appearance 
# -------------------------------------

# Merge report definition header with report
report = pd.merge(definition_df, report, left_on="definition_id", right_on="definition_id", how="left")

# Sort based on key value
report.sort_values("key", inplace=True)


In [252]:
report

,definition_id,key,definition_type,name,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00,2024-05-31 00:00:00,2024-06-30 00:00:00,...,2025-03-31 00:00:00,2025-04-30 00:00:00,2025-05-31 00:00:00,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
0,1,100,NaN,PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,50231.12,42880.14,26308.59,38379.57,39246.76,19862.60,...,29564.02,31612.94,1223518.79,78614.91,50383.77,109618.83,45848.99,79478.46,64314.40,78589.64
2,3,102,1.0,Maksājumi piegādātājiem,-30281.91,-43778.47,-56641.43,-36223.79,-53628.12,-41573.54,...,-33160.82,-25183.06,-1412305.45,-53911.59,-71354.35,-65696.15,-57152.91,-83340.54,-87955.73,-85636.35
3,4,103,1.0,Maksājumi darbiniekiem,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,39997.26,11550.30,-3411.93,6743.59,2071.27,9750.12,...,-10285.45,-6691.37,472238.27,32183.35,23817.56,23586.95,16967.53,52095.60,6595.89,50901.75
5,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,53707.79,71.62,-45278.94,1911.90,-55277.69,-34330.78,...,-20971.84,-15754.08,-172783.36,20090.83,-18257.62,50367.11,-32641.26,5430.00,-52211.13,23624.18
6,7,201,1.0,Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,8,202,1.0,Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,53707.79,71.62,-45278.94,1911.90,-55277.69,-34330.78,...,-20971.84,-15754.08,-172783.36,20090.83,-18257.62,50367.11,-32641.26,5430.00,-52211.13,23624.18


In [253]:
# Subtract report formatting in separate dataframe
format_df = report["definition_type"]

In [254]:
format_df

0     NaN
1     1.0
2     1.0
3     1.0
4     1.0
5     2.0
6     1.0
7     1.0
8     1.0
9     2.0
10    NaN
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    2.0
20    NaN
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    2.0
27    1.0
28    2.0
29    3.0
Name: definition_type, dtype: float64

In [255]:
# Prepare report_df for visual appearance

report.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
report.set_index("name", inplace=True)

# Format column headers to show only the date part
report.columns = [col.strftime(date_format()) if not pd.isnull(col) else col for col in report.columns]

In [256]:
report

,"31. Jan, 2024","29. Feb, 2024","31. Mar, 2024","30. Apr, 2024","31. May, 2024","30. Jun, 2024","31. Jul, 2024","31. Aug, 2024","30. Sep, 2024","31. Oct, 2024",...,"31. Mar, 2025","30. Apr, 2025","31. May, 2025","30. Jun, 2025","31. Jul, 2025","31. Aug, 2025","30. Sep, 2025","31. Oct, 2025","30. Nov, 2025","31. Dec, 2025"
name,,,,,,,,,,,,,,,,,,,,,
PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ieņēmumi no preču un pakalpojumu pārdošanas,50231.12,42880.14,26308.59,38379.57,39246.76,19862.60,28071.46,48333.03,19896.11,41910.53,...,29564.02,31612.94,1223518.79,78614.91,50383.77,109618.83,45848.99,79478.46,64314.40,78589.64
Maksājumi piegādātājiem,-30281.91,-43778.47,-56641.43,-36223.79,-53628.12,-41573.54,-27744.61,-41369.98,-37040.46,-29564.45,...,-33160.82,-25183.06,-1412305.45,-53911.59,-71354.35,-65696.15,-57152.91,-83340.54,-87955.73,-85636.35
Maksājumi darbiniekiem,-6238.68,-10580.35,-11534.17,-6987.47,-42967.60,-22369.96,-3343.80,-6555.90,-6808.37,-6539.37,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
Pārējie pamatdarbības ieņēmumi un izdevumi,39997.26,11550.30,-3411.93,6743.59,2071.27,9750.12,11096.88,-11234.59,-5011.91,7518.95,...,-10285.45,-6691.37,472238.27,32183.35,23817.56,23586.95,16967.53,52095.60,6595.89,50901.75
BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,53707.79,71.62,-45278.94,1911.90,-55277.69,-34330.78,8079.93,-10827.44,-28964.63,13325.66,...,-20971.84,-15754.08,-172783.36,20090.83,-18257.62,50367.11,-32641.26,5430.00,-52211.13,23624.18
Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
